In [1]:
import numpy as np
import utils
import torch
import pandas as pd
import gpytorch

from tqdm.notebook import tqdm
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel

%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Thu Feb 16 21:01:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:21:00.0 Off |                  Off |
| 30%   38C    P8    24W / 300W |      3MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:22:00.0 Off |                  Off |
| 49%   

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda', index=0)

In [58]:
class DirichletGPModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train(train_x, train_y, device):
    likelihood = DirichletClassificationLikelihood(train_y, learn_additional_noise=True)
    model = DirichletGPModel(train_x, likelihood.transformed_targets, likelihood, num_classes=likelihood.num_classes)
    
    model.to(device)
    likelihood.to(device)
    
    training_iterations = 50
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()
    
    for i in tqdm(range(training_iterations)):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y).sum()
        loss.backward()
        optimizer.step()
    
    return model.float(), likelihood.float()

In [59]:
dfx, dfy, cols = utils.get_dataset('adult_income', return_dataframe=True)

In [60]:
from sklearn.preprocessing import minmax_scale

In [61]:
minmax_separate = False

In [62]:
if minmax_separate:
    dfx_1 = dfx.loc[dfx.gender == 1]
    dfx_0 = dfx.loc[dfx.gender == 0]
    dfx_1[cols] = minmax_scale(dfx_1[cols])
    dfx_0[cols] = minmax_scale(dfx_0[cols])
else:
    dfx[cols] = minmax_scale(dfx[cols])
    dfx_1 = dfx.loc[dfx.gender == 1]
    dfx_0 = dfx.loc[dfx.gender == 0]
    dfy_1 = dfy.loc[dfx.gender == 1]
    dfy_0 = dfy.loc[dfx.gender == 0]

In [63]:
tensor_data_x0 = torch.tensor(dfx_0.values).to(device)
tensor_data_y0 = torch.tensor(dfy_0.values).to(device)
tensor_data_x1 = torch.tensor(dfx_1.values).to(device)
tensor_data_y1 = torch.tensor(dfy_1.values).to(device)

In [64]:
tensor_data_x0.shape

torch.Size([31114, 102])

In [65]:
import utils

In [66]:
warm_start_samples_0 = utils.sample_from_tensor(tensor_data_x0.shape[0], 500, device).long()
warm_start_samples_1 = utils.sample_from_tensor(tensor_data_x1.shape[0], 500, device).long()

In [67]:
non_warm_start_samples_0 = utils.complement_idx(warm_start_samples_0, tensor_data_x0.shape[0])
non_warm_start_samples_1 = utils.complement_idx(warm_start_samples_1, tensor_data_x1.shape[0])

In [68]:
warm_start_x0, warm_start_y0 = tensor_data_x0[warm_start_samples_0], tensor_data_y0[warm_start_samples_0]
warm_start_x1, warm_start_y1 = tensor_data_x1[warm_start_samples_1], tensor_data_y1[warm_start_samples_1]

In [69]:
warm_start_y0.float().mean() - warm_start_y1.float().mean()

tensor(0.2060, device='cuda:0')

In [70]:
tensor_data_x0, tensor_data_y0 = tensor_data_x0[non_warm_start_samples_0], tensor_data_y0[non_warm_start_samples_0]
tensor_data_x1, tensor_data_y1 = tensor_data_x1[non_warm_start_samples_1], tensor_data_y1[non_warm_start_samples_1]

In [71]:
model0, likelihood0 = train(warm_start_x0, warm_start_y0, device)
model1, likelihood1 = train(warm_start_x1, warm_start_y1, device)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [72]:
model0.eval(), model1.eval(), likelihood0.eval(), likelihood1.eval();

In [73]:
from vae_models import VanillaVAE as vae

In [74]:
vae0 = vae()
vae0.load_state_dict(torch.load("checkpoints/adult_income/vae_xA=0/best.pt"))
vae0.eval()
vae0.to(device);

In [75]:
vae1 = vae()
vae1.load_state_dict(torch.load("checkpoints/adult_income/vae_xA=1/best.pt"))
vae1.eval()
vae1.to(device);

In [76]:
from blackbox_models import BlackBox

In [77]:
blackbox = BlackBox('Logistic', 102, 1)
blackbox.load_state_dict(torch.load("/mnt/infonas/data/eeshaan/fairness/EE492/checkpoints/adult_income/blackbox/Logistic/best.pt"))
blackbox.eval()
blackbox.to(device);

In [78]:
k_nearby_points = 100
total_selection = 4000

In [79]:
tensor_data_y0.float().mean() - tensor_data_y1.float().mean()

tensor(0.1989, device='cuda:0')

In [27]:
s = utils.sample_randomly_from_input_space('adult_income').to(device) 
s.dtype

torch.float32

First point -> either sample from the first 1000 random points
or sample from the input space

Q = 5000
Qi = 1000

4000 <- (x' <- random)

In [186]:
lambda_reg = 1.0
neg_queried = warm_start_x0.clone()
neg_labels = warm_start_y0.clone().unsqueeze(1)

for epoch_outer in tqdm(range(total_selection // k_nearby_points)):
    s = utils.sample_randomly_from_input_space('adult_income').to(device) 
    
    optimizer0 = torch.optim.AdamW((s,), lr=1e-5)
    best_loss = 10e5
    count = 0
    losses = []
    for epoch in range(1,1000):
        optimizer0.zero_grad()
        x0_samples = utils.postprocess(
            vae0.sample(s.to(device), 100, device).squeeze(1), 'adult_income').float()

        var_term = likelihood0(model0(x0_samples)).variance.sum(axis=0).mean()
        ll_term = 0 #- lambda_reg * ((s - x0_samples)**2).mean()
        obj0 =  var_term + ll_term

        loss = -obj0
        print(loss)
        loss.backward()
        optimizer0.step()
        if loss < best_loss:
            best_loss = loss
            count = 0
            losses.append(loss)
        else:
            count += 1
        if count  == 50:
            break
    q, tensor_data_x0, tensor_data_y0 = utils.query_nearby(s, tensor_data_x0, tensor_data_y0, k_nearby_points)
    if epoch_outer == 0:
        candidates = q
    else:
        candidates = torch.cat((candidates, q), dim=0)

    
    # if epoch_outer % 100 == 0:
    #     #new_vals = torch.concatenate(candidates)
    #     new_queries,_, _ = vae0(new_vals.to(device))
    #     new_labels = blackbox(new_queries)
    #     neg_queried = torch.concatenate([neg_queried, new_queries])
    #     neg_labels = torch.concatenate([neg_labels, (0.5*(torch.sign(new_labels - 0.5) + 1.0)).long().detach().clone()])
    #     model0, likelihood0 = train(neg_queried.detach().clone(),neg_labels.flatten(), device)
    #     model0.eval()
    #     likelihood0.eval()
    #     candidates = []

  0%|          | 0/40 [00:00<?, ?it/s]

In [135]:
model0

DirichletGPModel(
  (likelihood): DirichletClassificationLikelihood(
    (noise_covar): FixedGaussianNoise()
    (second_noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)

In [138]:
for x in model0.parameters():
    print(x.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


In [139]:
model0(x0_samples)

RuntimeError: expected scalar type Float but found Double

In [20]:
lambda_reg = 1.0
candidates = []
neg_queried = warm_start_x0.clone()
neg_labels = warm_start_y0.clone().unsqueeze(1)

for epoch_outer in tqdm(range(1, 4001)):
    x0_random = torch.normal(0.,1.,size=(1,102), dtype=torch.float32, requires_grad=True)
    optimizer0 = torch.optim.AdamW((x0_random,), lr=1.0)
    best_loss = 10e5
    count = 0
    losses = []
    for epoch in range(1,100):
        optimizer0.zero_grad()
        x0_samples = utils.postprocess(
            vae0.sample(x0_random.to(device), 100, device, 
                        **{'tau': 1.0, 'tau_min': 0.1, 'anneal_rate': 3e-5, 'steps': 0, 'hard': False}).squeeze(1),
            'adult_income'
        )
        obj0 = likelihood0(model0(x0_samples.to(device))).variance.sum(axis=0).mean() - lambda_reg * ((x0_random.to(device) - x0_samples)**2).mean()
#         print(obj0.shape)
        loss = -obj0
        loss.backward()
        optimizer0.step()
        if loss < best_loss:
            best_loss = loss
            count = 0
            losses.append(loss)
        else:
            count += 1
        if count  == 5:
            break
    x0_query = dfx_0.iloc[np.argmin(np.linalg.norm(np.array(dfx_0) - x0_random.detach().clone().numpy()))]
    candidates.append(torch.Tensor([x0_query]).float().to(device))
    
    if epoch_outer % 100 == 0:
        new_vals = torch.concatenate(candidates)
        new_queries,_, _ = vae0(new_vals.to(device), **{'tau': 1.0, 'tau_min': 0.1, 'anneal_rate': 3e-5, 'steps': 0, 'hard': False})
        new_labels = blackbox(new_queries)
        neg_queried = torch.concatenate([neg_queried, new_queries])
        neg_labels = torch.concatenate([neg_labels, (0.5*(torch.sign(new_labels - 0.5) + 1.0)).long().detach().clone()])
        print(neg_labels.shape, neg_labels.dtype)
        print(neg_queried.shape, neg_queried.dtype)
        model0, likelihood0 = train(neg_queried.detach().clone(),neg_labels.flatten())
        model0.eval()
        likelihood0.eval()
        candidates = []

  0%|          | 0/4000 [00:00<?, ?it/s]

torch.Size([1100, 1]) torch.int64
torch.Size([1100, 102]) torch.float32


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([1200, 1]) torch.int64
torch.Size([1200, 102]) torch.float32


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([4900, 1]) torch.int64
torch.Size([4900, 102]) torch.float32


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([5000, 1]) torch.int64
torch.Size([5000, 102]) torch.float32


  0%|          | 0/50 [00:00<?, ?it/s]

In [21]:
candidates = []
pos_queried = warm_start_x1.clone()
pos_labels = warm_start_y1.clone().unsqueeze(1)
print(pos_labels.shape, pos_labels.dtype)
print(pos_queried.shape, pos_queried.dtype)

for epoch_outer in tqdm(range(1, 4001)):
    x1_random = torch.normal(0.,1.,size=(1,102), dtype=torch.float32, requires_grad=True)
    optimizer1 = torch.optim.AdamW((x1_random,), lr=1.0)
    best_loss = 10e5
    count = 0
    losses = []
    for epoch in range(1,100):
        optimizer1.zero_grad()
        x1_samples = utils.postprocess(
            vae1.sample(x1_random.to(device), 100, device, 
                        **{'tau': 1.0, 'tau_min': 0.1, 'anneal_rate': 3e-5, 'steps': 0, 'hard': False}).squeeze(1),
            'adult_income'
        )
        obj1 = likelihood1(model1(x1_samples.to(device))).variance.sum(axis=0).mean() - lambda_reg * ((x1_random.to(device) - x1_samples)**2).mean()
#         print(obj0.shape)
        loss = -obj1
        loss.backward()
        optimizer1.step()
        if loss < best_loss:
            best_loss = loss
            count = 0
            losses.append(loss)
        else:
            count += 1
        if count  == 5:
            break
    x1_query = dfx_1.iloc[np.argmin(np.linalg.norm(np.array(dfx_1) - x1_random.detach().clone().numpy()))]
    candidates.append(torch.Tensor([x1_query]).float().to(device))
    
    if epoch_outer % 100 == 0:
        new_vals = torch.concatenate(candidates)
        new_queries,_ , _ = vae1(new_vals.to(device), **{'tau': 1.0, 'tau_min': 0.1, 'anneal_rate': 3e-5, 'steps': 0, 'hard': False})
        new_labels = blackbox(new_queries)
        pos_queried = torch.concatenate([pos_queried, new_queries])
        pos_labels = torch.concatenate([pos_labels, (0.5*(torch.sign(new_labels - 0.5) + 1.0)).long().detach().clone()])
        model1, likelihood1 = train(pos_queried.detach().clone(),pos_labels.flatten())
        model1.eval()
        likelihood1.eval()
        candidates = []

torch.Size([1000, 1]) torch.int64
torch.Size([1000, 102]) torch.float32


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [22]:
pos_labels.shape

torch.Size([5000, 1])

In [23]:
new_labels.shape

torch.Size([100, 1])

In [24]:
# parity in data
np.abs(dfy[dfx[dfx.gender == 0].index].mean() - dfy[dfx[dfx.gender == 1].index].mean())

0.19911019753072282

In [25]:
bb_input0 = torch.from_numpy(dfx_0.values).float().to(device)
bb_input1 = torch.from_numpy(dfx_1.values).float().to(device)

In [26]:
y0 = blackbox(bb_input0)
y1 = blackbox(bb_input1)

In [27]:
y0_ = torch.round(y0)
y1_ = torch.round(y1)

In [28]:
y0_.sum()

tensor(7659., device='cuda:0', grad_fn=<SumBackward0>)

In [29]:
torch.abs(y0_.mean() - y1_.mean())

tensor(0.1869, device='cuda:0', grad_fn=<AbsBackward0>)

In [35]:
dp_value = torch.abs(pos_labels.squeeze().float().mean() - neg_labels.squeeze().float().mean())
dp_value

tensor(0.0406, device='cuda:0')

In [36]:
pos_labels.float().mean()

tensor(0.0196, device='cuda:0')

In [37]:
neg_labels.float().mean()

tensor(0.0602, device='cuda:0')

In [39]:
torch.save(dp_value, 'results/adult_income/dp_value_vanilla_vae_reg.pt')